# Test Load Daily

Start by loading one "new" tile.

## Imports

In [1]:
import os
from glob import glob
from pytz import utc
import numpy as np
from astropy import __version__ as astropy_version
from astropy.io import fits
from astropy.table import Table, Column, join
from astropy.time import Time
from sqlalchemy import __version__ as sqlalchemy_version
from desiutil import __version__ as desiutil_version
from desiutil.names import radec_to_desiname
from desiutil.log import get_logger, DEBUG
from desispec.io.photo import gather_tractorphot, gather_targetphot
from desispec.io.meta import findfile
from desispec.scripts.zcatalog import read_redrock
from desispec.zcatalog import find_primary_spectra
from specprodDB import __version__ as specprodDB_version
import specprodDB.load as db
from specprodDB.util import cameraid, spgrpid, surveyid, no_sky

## Initial Values

In [2]:
specprod = os.environ['SPECPROD']
# tile_id, tile_survey, tile_program = 3867, 'main', 'dark'
# tile_id, tile_survey, tile_program = 5053, 'main', 'dark'
# tile_id, tile_survey, tile_program = 5052, 'main', 'dark'
# tile_id, tile_survey, tile_program = 5074, 'main', 'dark'
# tile_id, tile_survey, tile_program = 1685, 'main', 'dark'
# tile_id, tile_survey, tile_program = 40069, 'main', 'backup'
tile_id, tile_survey, tile_program = 80950, 'sv1', 'backup'
overwrite = False

## Initialize Database

In [3]:
db.log = get_logger(DEBUG)
postgresql = db.setup_db(schema=specprod, hostname='db-loadbalancer.bweaver.development.svc.spin.nersc.org', username='desi_admin', overwrite=overwrite)
if overwrite:
    versions = [db.Version(package='specprod-db', version=specprodDB_version),
                db.Version(package='lsdr9-photometry', version='main'), # Maybe 'desispec' to mean computed rather than fetched from files?
                db.Version(package='redshift', version='daily/v0'),
                db.Version(package='tiles', version='trunk'),
                db.Version(package='specprod', version=specprod),
                db.Version(package='numpy', version=np.__version__),
                db.Version(package='astropy', version=astropy_version),
                db.Version(package='sqlalchemy', version=sqlalchemy_version),
                db.Version(package='desiutil', version=desiutil_version)]
    db.dbSession.add_all(versions)
    db.dbSession.commit()

## Read tiles file

In [4]:
# tiles_file = findfile('tiles', readonly=True).replace('.fits', '.csv')
tiles_file = os.path.join(os.environ['SCRATCH'], 'tiles-daily-patched-with-jura.csv')
tiles_table = Table.read(tiles_file, format='ascii.csv')
# tiles_table

In [5]:
tiles_table[tiles_table['TILEID'] == tile_id]

TILEID,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,NEXP,EXPTIME,TILERA,TILEDEC,EFFTIME_ETC,EFFTIME_SPEC,EFFTIME_GFA,GOALTIME,OBSSTATUS,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GOALTYPE,MINTFRAC,LASTNIGHT
int64,str7,str6,str16,str19,int64,float64,float64,float64,float64,float64,float64,float64,str8,float64,float64,float64,float64,str7,float64,int64
80950,sv1,backup,backup1,sv1backup1,1,1820.1,180.0,70.0,0.0,349.7,320.0,30.0,obsstart,283.4,349.7,278.7,176.7,backup,0.9,20210422


In [6]:
row_index = np.where(tiles_table['TILEID'] == tile_id)[0]
assert (tiles_table[row_index]['LASTNIGHT'] >= 20201214).all()
try:
    missing_program = tiles_table[row_index[0]]['PROGRAM'].mask
except AttributeError:
    missing_program = False
if missing_program:
    print("WARNING: Replacing missing value for 'PROGRAM'!")
    tiles_table['PROGRAM'][row_index] = tile_program
    tiles_table['PROGRAM'][row_index].mask = False
load_tiles = db.Tile.convert(tiles_table, row_index=row_index)
assert len(load_tiles) == 1
new_tile = load_tiles[0]

In [7]:
# db.dbSession.rollback()
db.dbSession.add_all(load_tiles)
db.dbSession.commit()

## Read exposures file

The daily exposures file may contain exposures with `EFFTIME_SPEC == 0`. We do not want to load these.

In [8]:
# exposures_file = findfile('exposures', readonly=True)
exposures_file = os.path.join(os.environ['SCRATCH'], 'exposures-daily-patched-with-jura.fits')
exposures_table = Table.read(exposures_file, format='fits', hdu='EXPOSURES')
frames_table = Table.read(exposures_file, format='fits', hdu='FRAMES')
exposures_table[exposures_table['TILEID'] == new_tile.tileid]

NIGHT,EXPID,TILEID,TILERA,TILEDEC,MJD,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,EXPTIME,EFFTIME_SPEC,GOALTIME,GOALTYPE,MINTFRAC,AIRMASS,EBV,SEEING_ETC,EFFTIME_ETC,TSNR2_ELG,TSNR2_QSO,TSNR2_LRG,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBDARK,TSNR2_GPBBRIGHT,TSNR2_GPBBACKUP,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GPB_EFFTIME_DARK,GPB_EFFTIME_BRIGHT,GPB_EFFTIME_BACKUP,TRANSPARENCY_GFA,SEEING_GFA,FIBER_FRACFLUX_GFA,FIBER_FRACFLUX_ELG_GFA,FIBER_FRACFLUX_BGS_GFA,FIBERFAC_GFA,FIBERFAC_ELG_GFA,FIBERFAC_BGS_GFA,AIRMASS_GFA,SKY_MAG_AB_GFA,SKY_MAG_G_SPEC,SKY_MAG_R_SPEC,SKY_MAG_Z_SPEC,EFFTIME_GFA,EFFTIME_DARK_GFA,EFFTIME_BRIGHT_GFA,EFFTIME_BACKUP_GFA
int32,int32,int32,float64,float64,float64,bytes7,bytes6,bytes19,bytes19,float64,float64,float64,bytes7,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
20210422,86009,80950,180.0,70.0,59327.30637428,sv1,backup,backup1,sv1backup1,1820.0631103515625,349.74223480224623,0.0,backup,0.9,1.3359730243682861,0.016651306301355362,0.0,0.0,40.667701721191406,8.79162311553955,23.32752227783203,15.450385617040512,1990.453369140625,0.0,0.0,0.0,283.44352569811053,349.74223480224623,278.6634716796875,176.74867293683263,0.0,0.0,0.0,0.9963640063948922,1.414504738104407,0.42216596704352016,0.3219042844043313,0.15271744880763222,0.6617699344165768,0.7025754164711452,0.7286478535891991,1.3554805696210628,19.449657460646147,19.205123448622775,19.306335359842933,18.76489994729587,320.03049728256417,213.57355830447153,271.071070751922,320.03049728256417


In [9]:
row_index = np.where((exposures_table['TILEID'] == new_tile.tileid) & (exposures_table['EFFTIME_SPEC'] > 0))[0]
assert len(row_index) > 0
load_exposures = db.Exposure.convert(exposures_table, row_index=row_index)
# load_exposures

In [10]:
# db.dbSession.rollback()
db.dbSession.add_all(load_exposures)
db.dbSession.commit()

In [11]:
load_frames = list()
for exposure in load_exposures:
    row_index = np.where(frames_table['EXPID'] == exposure.expid)[0]
    assert len(row_index) > 0
    load_frames += db.Frame.convert(frames_table, row_index=row_index)
# load_frames

In [12]:
# db.dbSession.rollback()
db.dbSession.add_all(load_frames)
db.dbSession.commit()

## Load photometry for the tile

When tractor photometry is written out by John Moustakas' VAC code, only objects with `brickname != ''` are written.

Should filter on TARGETID already loaded before creating the `potential_cat`.

In [13]:
fiberassign_file = findfile('fiberassignsvn', tile=new_tile.tileid, readonly=True)
potential_targets_table = Table.read(fiberassign_file, format='fits', hdu='TARGETS')
no_sky_rows = no_sky(potential_targets_table)
potential_cat = Table()
potential_cat['TARGETID'] = potential_targets_table['TARGETID'][no_sky_rows]
potential_cat['TILEID'] = new_tile.tileid
potential_cat['TARGET_RA'] = potential_targets_table['RA'][no_sky_rows]
potential_cat['TARGET_DEC'] = potential_targets_table['DEC'][no_sky_rows]
# potential_cat['PETAL_LOC'] = potential_targets_table['PETAL_LOC'][no_sky_rows]
potential_cat['SURVEY'] = new_tile.survey
potential_cat['PROGRAM'] = new_tile.program
# potential_cat

In [14]:
potential_targetphot = gather_targetphot(potential_cat, racolumn='TARGET_RA', deccolumn='TARGET_DEC')
potential_targetphot['SURVEY'] = potential_cat['SURVEY']
potential_targetphot['PROGRAM'] = potential_cat['PROGRAM']
potential_targetphot['TILEID'] = potential_cat['TILEID']
# potential_targetphot['DESINAME'] = radec_to_desiname(potential_targetphot['RA'], potential_targetphot['DEC'])
inan = np.logical_or(np.isnan(potential_targetphot['PMRA']), np.isnan(potential_targetphot['PMDEC']))
if np.any(inan):
    potential_targetphot['PMRA'][inan] = 0.0
    potential_targetphot['PMDEC'][inan] = 0.0
# potential_targetphot

In [15]:
potential_tractorphot = gather_tractorphot(potential_cat, racolumn='TARGET_RA', deccolumn='TARGET_DEC')

In [16]:
assert (np.where(potential_tractorphot['RELEASE'] == 0)[0] == np.where(potential_tractorphot['BRICKNAME'] == '')[0]).all()

In [17]:
potential_tractorphot_already_loaded = db.dbSession.query(db.Photometry.targetid).filter(db.Photometry.targetid.in_(potential_tractorphot['TARGETID'].tolist())).all()
potential_tractorphot_not_already_loaded = np.ones((len(potential_tractorphot),), dtype=bool)
for row in potential_tractorphot_already_loaded:
    potential_tractorphot_not_already_loaded[potential_tractorphot['TARGETID'] == row[0]] = False

In [18]:
row_index = np.where((potential_tractorphot['BRICKNAME'] != '') & potential_tractorphot_not_already_loaded)[0]
load_photometry = db.Photometry.convert(potential_tractorphot, row_index=row_index)
# load_photometry[:20]

In [19]:
if len(load_photometry) > 0:
    # db.dbSession.rollback()
    db.dbSession.add_all(load_photometry)
    db.dbSession.commit()

### Load photometry, such as it is, for objects that are not in the tractor catalog

In [20]:
load_rows = np.zeros((len(potential_targetphot),), dtype=bool)
if len(load_photometry) > 0:
    loaded_targetid = Table()
    loaded_targetid['TARGETID'] = np.array([r.targetid for r in load_photometry])
    loaded_targetid['LS_ID'] = np.array([r.ls_id for r in load_photometry])
    j = join(potential_targetphot['TARGETID', 'RELEASE'], loaded_targetid, join_type='left', keys='TARGETID')
    try:
        load_targetids = j['TARGETID'][j['LS_ID'].mask]
    except AttributeError:
        #
        # This means *every* TARGETID is already loaded.
        #
        pass
    else:
        unique_targetid, targetid_index = np.unique(potential_targetphot['TARGETID'].data, return_index=True)
        for t in load_targetids:
            load_rows[targetid_index[unique_targetid == t]] = True
# load_rows

In [21]:
potential_targetphot_already_loaded = db.dbSession.query(db.Photometry.targetid).filter(db.Photometry.targetid.in_(potential_targetphot[load_rows]['TARGETID'].tolist())).all()
potential_targetphot_not_already_loaded = np.ones((len(potential_targetphot),), dtype=bool)
for row in potential_targetphot_already_loaded:
    potential_targetphot_not_already_loaded[potential_targetphot['TARGETID'] == row[0]] = False

In [22]:
row_index = np.where(load_rows & potential_targetphot_not_already_loaded)[0]
load_targetphot = db.Photometry.convert(potential_targetphot, row_index=row_index)
# load_targetphot[:20]

In [23]:
if len(load_targetphot) > 0:
    # db.dbSession.rollback()
    db.dbSession.add_all(load_targetphot)
    db.dbSession.commit()

In [24]:
load_target = db.Target.convert(potential_targetphot, new_tile.survey, new_tile.tileid)
# load_target[:20]

In [25]:
# db.dbSession.rollback()
db.dbSession.add_all(load_target)
db.dbSession.commit()

## Load tile/cumulative redshifts

Need a way to compute "best" spectra as new tiles are added. There are a lot of columns that come from other sources here, so need to track these down.

In [26]:
redrock_files = list()
for spectrograph in range(10):
    redrock_file, redrock_exists = findfile('redrock_tile', groupname='cumulative', tile=new_tile.tileid, spectrograph=spectrograph, night=new_tile.lastnight, readonly=True, return_exists=True)
    if redrock_exists:
        redrock_files.append(redrock_file)

In [27]:
load_ztile = list()
for rr in redrock_files:
    redrock_table, expfibermap = read_redrock(rr, group='cumulative', recoadd_fibermap=True, pertile=True)
    firstnight = np.zeros(len(redrock_table), dtype=np.int32)
    row_index = no_sky(redrock_table)
    if len(row_index) > 0:
        data_columns = list()
        for c in db.Ztile.__table__.columns:
            if c.name == 'id':
                id0 = spgrpid('cumulative') << 27 | redrock_table['SPGRPVAL'][row_index].base.astype(np.int64)
                data_columns.append([(i0 << 64) | i1 for i0, i1 in zip(id0.tolist(), redrock_table['TARGETID'][row_index].tolist())])
            elif c.name == 'targetphotid':
                id0 = np.array([surveyid(new_tile.survey) << 32 | new_tile.tileid]*len(row_index), dtype=np.int64)
                data_columns.append([(i0 << 64) | i1 for i0, i1 in zip(id0.tolist(), redrock_table['TARGETID'][row_index].tolist())])
            elif c.name == 'spgrp':
                data_columns.append(['cumulative']*len(row_index))
            elif c.name == 'desiname':
                data_columns.append(radec_to_desiname(redrock_table['TARGET_RA'][row_index], redrock_table['TARGET_DEC'][row_index]).tolist())
            elif c.name == 'survey':
                data_columns.append([new_tile.survey]*len(row_index))
            elif c.name == 'program':
                data_columns.append([new_tile.program]*len(row_index))
            elif c.name == 'firstnight':
                for tilefm in Table(expfibermap[['TILEID', 'NIGHT']]).group_by('TILEID').groups:
                    tileid = tilefm['TILEID'][0]
                    iitile = redrock_table['TILEID'] == new_tile.tileid
                    firstnight[iitile] = np.min(tilefm['NIGHT'])
                assert (firstnight != 0).all()
                data_columns.append(firstnight[row_index].tolist())
            elif c.name in ('sv_nspec', 'main_nspec', 'zcat_nspec'):
                data_columns.append([0]*len(row_index))
            elif c.name in ('sv_primary', 'main_primary', 'zcat_primary'):
                data_columns.append([False]*len(row_index))
            # elif c.name in ('min_mjd', 'mean_mjd', 'max_mjd'):
            #     data_columns.append([0.0]*len(row_index))
            elif c.name.startswith('coeff_'):
                coeff_index = int(c.name.split('_')[1])
                data_columns.append(redrock_table[row_index]['COEFF'][:, coeff_index].tolist())
            else:
                data_columns.append(redrock_table[row_index][c.name.upper()].tolist())
    data_rows = list(zip(*data_columns))
    load_ztile += [db.Ztile(**(dict([(c.name, d) for c, d in zip(db.Ztile.__table__.columns, row)]))) for row in data_rows]

INFO:zcatalog.py:114:read_redrock: Reading /dvs_ro/cfs/cdirs/desi/spectro/redux/jura/tiles/cumulative/80950/20210422/redrock-0-80950-thru20210422.fits
INFO:zcatalog.py:128:read_redrock: Recoadding fibermap from spectra-0-80950-thru20210422.fits.gz
INFO:zcatalog.py:114:read_redrock: Reading /dvs_ro/cfs/cdirs/desi/spectro/redux/jura/tiles/cumulative/80950/20210422/redrock-1-80950-thru20210422.fits
INFO:zcatalog.py:128:read_redrock: Recoadding fibermap from spectra-1-80950-thru20210422.fits.gz
INFO:zcatalog.py:114:read_redrock: Reading /dvs_ro/cfs/cdirs/desi/spectro/redux/jura/tiles/cumulative/80950/20210422/redrock-2-80950-thru20210422.fits
INFO:zcatalog.py:128:read_redrock: Recoadding fibermap from spectra-2-80950-thru20210422.fits.gz
INFO:zcatalog.py:114:read_redrock: Reading /dvs_ro/cfs/cdirs/desi/spectro/redux/jura/tiles/cumulative/80950/20210422/redrock-3-80950-thru20210422.fits
INFO:zcatalog.py:128:read_redrock: Recoadding fibermap from spectra-3-80950-thru20210422.fits.gz
INFO:zca

In [28]:
# db.dbSession.rollback()
db.dbSession.add_all(load_ztile)
db.dbSession.commit()

## Load fiberassign and potential

In [29]:
fiberassign_table = Table.read(fiberassign_file, format='fits', hdu='FIBERASSIGN')
potential_table = Table.read(fiberassign_file, format='fits', hdu='POTENTIAL_ASSIGNMENTS')

In [30]:
row_index = no_sky(fiberassign_table)
load_fiberassign = db.Fiberassign.convert(fiberassign_table, new_tile.tileid, row_index=row_index)
# load_fiberassign

In [31]:
# db.dbSession.rollback()
db.dbSession.add_all(load_fiberassign)
db.dbSession.commit()

In [32]:
row_index = no_sky(potential_table)
load_potential = db.Potential.convert(potential_table, new_tile.tileid, row_index=row_index)
# load_potential

In [33]:
# db.dbSession.rollback()
db.dbSession.add_all(load_potential)
db.dbSession.commit()

## Recompute global values

The global values are the primary classification and number of spectra.

In [34]:
zall_tilecumulative = db.dbSession.query(db.Ztile).all()
zall_table = Table(list(zip(*[(z.targetid, z.zwarn, z.tsnr2_lrg) for z in zall_tilecumulative])), names=('TARGETID', 'ZWARN', 'TSNR2_LRG'))
nspec, primary = find_primary_spectra(zall_table)
zcat_nspec, zcat_primary = nspec.tolist(), primary.tolist()
for k, z in enumerate(zall_tilecumulative):
    z.zcat_nspec = zcat_nspec[k]
    z.zcat_primary = zcat_primary[k]
db.dbSession.commit()

In [35]:
sv_tilecumulative = db.dbSession.query(db.Ztile).filter(db.Ztile.survey.in_(('sv1', 'sv2', 'sv3'))).all()
if len(sv_tilecumulative) > 0:
    sv_table = Table(list(zip(*[(z.targetid, z.zwarn, z.tsnr2_lrg) for z in sv_tilecumulative])), names=('TARGETID', 'ZWARN', 'TSNR2_LRG'))
    nspec, primary = find_primary_spectra(sv_table)
    sv_nspec, sv_primary = nspec.tolist(), primary.tolist()
    for k, z in enumerate(sv_tilecumulative):
        z.sv_nspec = sv_nspec[k]
        z.sv_primary = sv_primary[k]
    db.dbSession.commit()

In [36]:
main_tilecumulative = db.dbSession.query(db.Ztile).filter(db.Ztile.survey.in_(('main', ))).all()
if len(main_tilecumulative) > 0:
    main_table = Table(list(zip(*[(z.targetid, z.zwarn, z.tsnr2_lrg) for z in main_tilecumulative])), names=('TARGETID', 'ZWARN', 'TSNR2_LRG'))
    nspec, primary = find_primary_spectra(main_table)
    main_nspec, main_primary = nspec.tolist(), primary.tolist()
    for k, z in enumerate(main_tilecumulative):
        z.main_nspec = main_nspec[k]
        z.main_primary = main_primary[k]
    db.dbSession.commit()

## q3c Update

`tile`, `exposure`, `photometry`, `fiberassign`

In [37]:
q3c_updates = {'tile': 'tilera', 'exposure': 'tilera', 'photometry': 'ra', 'fiberassign': 'target_ra'}
for table in q3c_updates:
    db.q3c_index(table, ra=q3c_updates[table])

INFO:load.py:1417:q3c_index: Creating q3c index on jura.tile.
INFO:load.py:1419:q3c_index: Finished q3c index on jura.tile.
INFO:load.py:1417:q3c_index: Creating q3c index on jura.exposure.
INFO:load.py:1419:q3c_index: Finished q3c index on jura.exposure.
INFO:load.py:1417:q3c_index: Creating q3c index on jura.photometry.
INFO:load.py:1419:q3c_index: Finished q3c index on jura.photometry.
INFO:load.py:1417:q3c_index: Creating q3c index on jura.fiberassign.
INFO:load.py:1419:q3c_index: Finished q3c index on jura.fiberassign.
